In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Package for handling logistic regression
from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Importing our data
data = pd.read_excel('Data\\table_8_offenses_known_to_law_enforcement_virginia_by_city_2013.xls', 
                     encoding='latin1', skiprows=list(range(4)))

# This column contains no data, so we're going to remove it
data.drop(['Rape\n(legacy\ndefinition)2'], axis = 1, inplace = True)

# Cleaning up our column names
data.columns = data.columns.str.replace('\n', ' ')
data = data.rename(index = str, columns={'Rape (revised definition)1': 'Rape'})
data.drop(['City'], axis = 1, inplace = True)
data = data.dropna()
data = data[:-1]

In [3]:
data.head()

,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson
0,8186.000,10.000,0.000,3.000,1.000,6.000,233.000,20.000,198.000,15.000,4.000
1,148519.000,258.000,5.000,21.000,118.000,114.000,2967.000,249.000,2427.000,291.000,13.000
2,3486.000,8.000,0.000,0.000,2.000,6.000,56.000,4.000,52.000,0.000,0.000
3,2223.000,2.000,0.000,2.000,0.000,0.000,27.000,6.000,19.000,2.000,0.000
4,1728.000,12.000,0.000,2.000,2.000,8.000,77.000,25.000,51.000,1.000,0.000


In [4]:
data.describe()

,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson
count,148.000,148.000,148.000,148.000,148.000,148.000,148.000,148.000,148.000,148.000,148.000
mean,20170.378,65.014,1.304,6.953,20.365,36.392,612.554,100.926,478.953,32.676,3.277
std,54170.809,201.049,4.796,18.482,72.990,111.979,1729.381,312.329,1325.700,115.662,11.514
min,112.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,1438.500,1.000,0.000,0.000,0.000,0.000,19.000,3.000,16.750,0.000,0.000
50%,3347.500,5.000,0.000,1.000,0.000,4.000,62.500,9.000,54.500,2.000,0.000
75%,12460.250,25.250,0.000,4.000,6.000,13.250,272.000,27.250,251.500,11.000,1.000
max,450687.000,1418.000,37.000,140.000,624.000,842.000,11226.000,2039.000,9374.000,938.000,99.000


In [5]:
# Define the training and test sizes.
trainsize = int(data.shape[0] / 2)

# Create a dataframe where property crimes exceed 100
y = pd.DataFrame()
y['Property crime'] = np.where(data['Property crime'] < 100, 0, 1)

# Drop property crime from the dataframe since that's the 
# variable that we want to predict
data.drop(['Property crime'], axis = 1, inplace = True)

In [6]:
# Construct the test and training sets
df_test = data.iloc[trainsize:, :].copy()
y_test = y.iloc[trainsize:, :].copy()
df_train = data.iloc[:trainsize, :].copy()
y_train = y.iloc[:trainsize, :].copy()

In [7]:
# Builds the regression model
def regressionModelBuilder(X, y):
    lr = LogisticRegression(C=1e9)
    fit = lr.fit(X, y)

    # Display
    print('Coefficients')
    print(fit.coef_)
    print(fit.intercept_)
    pred_y_sklearn = lr.predict(X)

    print('\n Accuracy by admission status')
    print(pd.crosstab(pred_y_sklearn, y))

    print('\n Percentage accuracy')
    print(lr.score(X, y))

In [8]:
regressionModelBuilder(df_test, y_test)

Coefficients
[[-1.93761902e-04  3.27444050e-03  7.55980026e-05  3.75424145e-04
   1.06023788e-03  1.76318048e-03  2.51948699e-04  1.96495106e-02
   1.07589737e-03 -3.41016019e-04]]
[-0.00137817]

 Accuracy by admission status
col_0  (P, r, o, p, e, r, t, y,  , c, r, i, m, e)
row_0                                            
0                                              19
1                                              55

 Percentage accuracy
0.7432432432432432


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
regressionModelBuilder(df_train, y_test)

Coefficients
[[ 1.40544758e-04 -5.90381538e-02  6.99888136e-02 -2.34005951e-01
   1.99728766e-01 -9.47497821e-02  4.72857597e-03  1.15471906e-02
  -1.41360155e-01 -3.57753928e-01]]
[-0.16655995]

 Accuracy by admission status
col_0  (P, r, o, p, e, r, t, y,  , c, r, i, m, e)
row_0                                            
0                                              42
1                                              32

 Percentage accuracy
0.6216216216216216


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
ridgeregr = linear_model.Ridge(alpha=10, fit_intercept=False) 
ridgeregr.fit(df_train, y_train)
print(ridgeregr.score(df_train, y_train))
origparams = ridgeregr.coef_[0]
print(origparams)

0.3762206850794799
[ 1.06013834e-05 -3.42599430e-02 -1.44434599e-01  4.39060684e-02
  4.65222375e-02  1.97463506e-02 -2.07694190e-04  2.12858458e-03
 -1.64975855e-02  7.81198966e-03]


In [11]:
# Plot a line for each parameter.
for var in varstoplot:
    plt.plot(estimates_df['lambda'], estimates_df[var])
    labels.append(var)
plt.xlabel('lambda')
plt.ylabel('Parameter estimate size')
plt.legend(labels)
plt.show()

NameError: name 'varstoplot' is not defined